In [1]:
# Testing how TF Checkpoints work

import tensorflow as tf
import numpy as np
np.set_printoptions(precision=3, suppress=True)

In [2]:
x = tf.ones([6,1])

In [3]:
class MyModel(tf.keras.Model):
    
    def __init__(self):
        super().__init__(self)
        self.encoder = Encoder()
        
    def __call__(self, x):
        return self.encoder(x)
        
class Encoder(tf.keras.Model):
    
    def __init__(self):
        super().__init__(self)
        self.s = tf.keras.Sequential([tf.keras.layers.Dense(10), tf.keras.layers.Dense(10)])
        
    def __call__(self, x):
        return self.s(x)
        
model = MyModel()
_ = model(x)

In [4]:
ckpt = tf.train.Checkpoint(kwd=model)
ckpt.save("ckpt")

'ckpt-1'

In [5]:
w = model.get_weights()
for w_i in w:
    print(w_i[0], w_i.shape)

[ 0.615 -0.729  0.109 -0.288 -0.709  0.383  0.441  0.624  0.257  0.255] (1, 10)
0.0 (10,)
[-0.214  0.504  0.141 -0.164  0.367  0.281 -0.115  0.459 -0.433 -0.485] (10, 10)
0.0 (10,)


In [7]:
model2 = MyModel()

In [11]:
ckpt2 = tf.train.Checkpoint(kwd=model2)
status = ckpt2.restore("ckpt-1")
_ = model2(x)

In [10]:
w = model2.get_weights()
for w_i in w:
    print(w_i[0], w_i.shape)

[ 0.615 -0.729  0.109 -0.288 -0.709  0.383  0.441  0.624  0.257  0.255] (1, 10)
0.0 (10,)
[-0.214  0.504  0.141 -0.164  0.367  0.281 -0.115  0.459 -0.433 -0.485] (10, 10)
0.0 (10,)


In [ ]:
status.assert_consumed()